# Transfer Learning with ResNet-50 on CIFAR-10 using Keras and TensorBoard

Transfer learning is a machine learning technique where a model developed for a task is reused as the starting point for a model on a second task. In this notebook, we'll demonstrate transfer learning using the ResNet-50 architecture on the CIFAR-10 dataset.

## Steps:
1. **Data Loading and Preprocessing:** We'll start by loading the CIFAR-10 dataset and preprocess the images to make them suitable for the ResNet-50 model.

2. **Model Building:** We'll use the ResNet-50 model pre-trained on the ImageNet dataset. We'll remove the top (classification) layer and add our custom layers suitable for CIFAR-10 classification.

3. **Training:** Initially, we'll freeze the layers of the ResNet-50 model and train only our custom layers. This is to prevent large gradient updates from ruining the pre-trained weights of the ResNet-50 model.

4. **Fine-tuning:** After initial training, we'll unfreeze some of the deeper layers of the ResNet-50 model and continue training. This allows our model to make minor adjustments to the deeper features to better suit our dataset.

5. **Monitoring with TensorBoard:** We'll use TensorBoard to monitor the training process. TensorBoard provides visual insights into the events happening during training, which can be helpful for debugging and optimization purposes.

Let's begin!

In [2]:
import numpy as np
import keras
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.callbacks import TensorBoard

#### Load Data

In [3]:
# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Preprocess the data
train_images = preprocess_input(train_images)
test_images = preprocess_input(test_images)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## Model Building

We'll now build our model. The steps are as follows:

1. **Load the ResNet-50 Model:** We'll use the ResNet-50 model pre-trained on the ImageNet dataset. This gives us a powerful feature extractor that we can leverage for our CIFAR-10 classification task.

2. **Add Custom Layers:** We'll add a global average pooling layer followed by a dense layer with 1024 units and ReLU activation. The final layer will be a dense layer with 10 units (for the 10 CIFAR-10 classes) and a softmax activation to output class probabilities.

3. **Compile the Model:** We'll compile the model using the Adam optimizer and the categorical crossentropy loss function. We'll also monitor accuracy during training.

In [4]:
# Load the ResNet-50 model with imagenet weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Training the Model

With our model built, we can now proceed to train it. We'll start by training only the custom layers we added, while keeping the ResNet-50 layers frozen. This is because the ResNet-50 layers already contain useful features (from being trained on ImageNet), and we don't want to modify them too much in the initial stages of training.

We'll also use TensorBoard to monitor our training in real-time. TensorBoard provides a suite of visualization tools to help understand, debug, and optimize the training process.

In [ ]:
# Resize images to 224x224 (required input size for ResNet-50)
train_images_resized = np.array([keras.preprocessing.image.array_to_img(img, scale=False).resize((224, 224)) for img in train_images])
test_images_resized = np.array([keras.preprocessing.image.array_to_img(img, scale=False).resize((224, 224)) for img in test_images])

# TensorBoard callback
tensorboard_callback = TensorBoard(log_dir='./logs/resnet', histogram_freq=1, write_graph=True, write_images=True)

# Train the model
model.fit(train_images_resized, train_labels, epochs=5, batch_size=32, validation_data=(test_images_resized, test_labels), callbacks=[tensorboard_callback])

## Fine-tuning the Model

After the initial training, we can proceed to fine-tune the model. Fine-tuning involves unfreezing some of the deeper layers of the ResNet-50 model and continuing the training. This allows the model to adjust the pre-trained features slightly to better fit our specific dataset.

For this demonstration, we'll unfreeze the last 5 layers of the ResNet-50 model and continue training. Remember to always use a lower learning rate during fine-tuning to ensure that the model converges smoothly.

In [ ]:
# Unfreeze the last 5 layers of the base model
for layer in base_model.layers[-5:]:
    layer.trainable = True

# Use a lower learning rate
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training the model
model.fit(train_images_resized, train_labels, epochs=5, batch_size=32, validation_data=(test_images_resized, test_labels), callbacks=[tensorboard_callback])

## Monitoring with TensorBoard

TensorBoard is a powerful tool provided by TensorFlow to visualize the training process. It offers a suite of visualization tools to understand, debug, and optimize the training.

To view the training progress and other insights, you can run TensorBoard by pointing it to the log directory we specified (`./logs`). This can be done using the command:

```
tensorboard --logdir=./logs
```

Once TensorBoard is running, you can navigate to the provided URL in your web browser to view the visualizations.

## Visual Comparison of Predictions

To visually compare the model's predictions with the actual labels, we'll use `matplotlib`. We'll select a few random images from the test set, make predictions using our trained model, and then plot the images with the predicted and actual labels side by side for comparison.

In [ ]:
import matplotlib.pyplot as plt

# Define CIFAR-10 class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Select a few random images from the test set
num_samples = 10
indices = np.random.choice(len(test_images_resized), size=num_samples, replace=False)
sample_images = test_images_resized[indices]
sample_labels = test_labels[indices]

# Make predictions using the model
predictions = model.predict(sample_images)
predicted_labels = np.argmax(predictions, axis=1)
actual_labels = np.argmax(sample_labels, axis=1)

# Plot the images with predicted and actual labels
plt.figure(figsize=(15, 5))
for i, (img, pred_label, actual_label) in enumerate(zip(sample_images, predicted_labels, actual_labels)):
    plt.subplot(2, num_samples // 2, i + 1)
    plt.imshow(img.astype('int32'))
    plt.title(f'Pred: {class_names[pred_label]}\nActual: {class_names[actual_label]}')
    plt.axis('off')
plt.tight_layout()
plt.show()